![](image.png)

## Criação do Prompt via UI

In [0]:
# Substitua o catálogo e o schema que irá usar e crie
CATALOG_NAME = "baraldi_catalog_new"
SCHEMA_NAME = "genai"

spark.sql(f"CREATE CATALOG IF NOT EXISTS {CATALOG_NAME}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG_NAME}.{SCHEMA_NAME}")

DataFrame[]

In [0]:
# Crie um experimento (aba Experiments), coloque um nome e vá na aba de prompts

![PromptUI.png](./PromptUI.png "PromptUI.png")

![PromptUI2.png](./PromptUI2.png "PromptUI2.png")

![PromptUI3.png](./PromptUI3.png "PromptUI3.png")

### Criação com Python SDK

In [0]:
# Garanta que o mlflow está atualizado para o 3.0
%pip install --upgrade mlflow

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
# Verifique se a versão do mlflow é pelo menos a 3.0
import mlflow
mlflow.VERSION

'3.4.0'

In [0]:
# Configure a tag de experimento
mlflow.set_experiment_tags({
    "mlflow.promptRegistryLocation": f"{CATALOG_NAME}.{SCHEMA_NAME}"
})

In [0]:
# Substitua com seu catalogo e schema
uc_schema = f"{CATALOG_NAME}.{SCHEMA_NAME}"

# Nomeie o prompt
prompt_name = "prompt_sumarizacao"

# Defina o prompt
initial_template = """\
Sumarize o conteúdo em {{num_sentences}} sentenças.

Conteúdo: {{content}}
"""

# Registre um novo prompt
prompt = mlflow.genai.register_prompt(
    name=f"{uc_schema}.{prompt_name}",
    template=initial_template,
    # Parametros opcionais
    commit_message="Initial version of summarization prompt",
    tags={
        "author": "data-science-team@company.com",
        "use_case": "document_summarization",
        "task": "summarization",
        "language": "pt-br",
        "model_compatibility": "modelo"
    }
)

print(f"Prompt criado '{prompt.name}' (version {prompt.version})")

Prompt criado 'baraldi_catalog_new.genai.prompt_sumarizacao' (version 1)


In [0]:
# Carregue uma versão específica
prompt = mlflow.genai.load_prompt(name_or_uri=f"prompts:/{uc_schema}.{prompt_name}/1")

# Ou use a sintaxe alternativa
prompt = mlflow.genai.load_prompt(name_or_uri=f"{uc_schema}.{prompt_name}", version="1")

In [0]:
# Importe as bibliotecas
import mlflow
from databricks.sdk import WorkspaceClient

In [0]:
# Instale a biblioteca da openai
%pip install openai

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
# Habilite o autolog
mlflow.openai.autolog()

# Configure o tracking
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment("/Users/daniel.baraldi@databricks.com/experimentoteste")

# Configure o client
w = WorkspaceClient()
client = w.serving_endpoints.get_open_ai_client()

# Selecione uma LLM
model_name = "databricks-claude-sonnet-4"

In [0]:
# Crie a aplicação
# Use o decorator para o entry point
@mlflow.trace
def my_app(content: str, num_sentences: int):
    # Formate com variáveis
    formatted_prompt = prompt.format(
        content=content,
        num_sentences=num_sentences
    )

    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {
                "role": "system",
                "content": "Você é um assistente útil e educado.",
            },
            {
                "role": "user",
                "content": formatted_prompt,
            },
        ],
    )
    return response.choices[0].message.content

result = my_app(content="""Pelo menos 214 mil imóveis na Região Metropolitana de São Paulo ainda estavam sem energia elétrica na manhã desta terça-feira (23) em razão do forte temporal da tarde de segunda (22).

A cidade de Cajamar é a mais prejudicada até o momento, com cerca de 21% dos imóveis sem luz. No total, havia cerca de 9,8 mil endereços no escuro na cidade até as ê7h da manhã, segundo o balanço da concessionária Enel.

As cidades de Diadema, Osasco, Cotia e Embu das Artes também têm mais milhares de imóveis sem luz. Em Osasco, também há 9,7 mil endereços sem fornecimento de energia, conforme a tabela abaixo.

A chuva que atingiu toda a Grande São Paulo também teve reflexo na operação no aeroporto de Congonhas, na Zona Sul de São Paulo, na manhã desta terça.

Segundo a Concessionária Aena, pelo menos quatro voos haviam sido cancelados no início da manhã em razão do mau tempo.

Os destinos dos voos eram para o Rio de Janeiro, Porto Alegre, Vitória e Salvador, das companhias aéreas Gol e Latam.

Na tarde de segunda, Congonhas sofreu com a chuva forte, que causou goteiras dentro da sala de embarque. Ao menos 13 pousos e 12 decolagens também foram cancelados, além de sete voos desviados para outros aeroportos, por causa do mau tempo.

Previsão para terça (23)
Essa terça-feira também será marcada por pancadas de chuva, segundo o Instituto Nacional de Meteorologia (Inmet). Os termômetros devem variar entre 14°C e 22°C. À noite, o céu deve ficar limpo.

Na segunda, a chuva causou transtornos em diferentes pontos da cidade. Na região do Pacaembu, Zona Oeste, próximo ao estádio, mansões ficaram destalhadas. No Brás, no Centro, parte do teto da estação da CPTM desabou. Passageiros registraram água da chuva caindo no saguão da estação, e o chão completamente molhado.


De acordo com o Inmet, a chuva deve seguir ao longo da semana, com previsão de pancadas na quarta (24) e na quinta (25).

Já na sexta (26) deve ser registrada a temperatura mais baixa da semana: mínima de 10°C, mas não deve chover.""", num_sentences=1)
result

'Um forte temporal atingiu a Região Metropolitana de São Paulo na segunda-feira (22), deixando 214 mil imóveis sem energia elétrica, causando cancelamentos de voos em Congonhas e diversos transtornos pela cidade, com previsão de mais chuvas ao longo da semana.'

Trace(trace_id=tr-3115465f572f345841cf7d9c2761945b)

In [0]:
# Defina um novo prompt
new_template = """\
Você é um expert em sumarização. Condense o conteúdo em exatamente {{ num_sentences }} sentença clara e informativa que captura pontos principais.

Conteúdo: {{content}}

Sua sumarização deve:
- Conter exatamente {{num_sentences}} sentença
- Incluir apenas a informação mais importante
- Ser escrito em tom neutro e objetivo
- Manter o mesmo nível de formalidade que o texto original
"""

# Registre a nova versão
updated_prompt = mlflow.genai.register_prompt(
    name=f"{uc_schema}.{prompt_name}",
    template=new_template,
    commit_message="Added detailed instructions for better output quality",
    tags={
        "author": "data-science-team@company.com",
        "improvement": "Added specific guidelines for summary quality"
    }
)

print(f"Criada versão {updated_prompt.version} of '{updated_prompt.name}'")

Criada versão 2 of 'baraldi_catalog_new.genai.prompt_sumarizacao'


In [0]:
# Carregue uma versão específica
prompt = mlflow.genai.load_prompt(name_or_uri=f"prompts:/{uc_schema}.{prompt_name}/2")

# Ou use outra sintaxe
prompt = mlflow.genai.load_prompt(name_or_uri=f"{uc_schema}.{prompt_name}", version="2")

In [0]:
# Busque prompts
results = mlflow.genai.search_prompts(f"catalog = '{CATALOG_NAME}' AND schema = '{SCHEMA_NAME}'")

# Use variáveis para o schema
catalog_name = uc_schema.split('.')[0] 
schema_name = uc_schema.split('.')[1]  
results = mlflow.genai.search_prompts(f"catalog = '{catalog_name}' AND schema = '{schema_name}'")

# Limite resultados
results = mlflow.genai.search_prompts(
    filter_string=f"catalog = '{catalog_name}' AND schema = '{schema_name}'",
    max_results=50
)
results

[<PromptInfo: name='baraldi_catalog_new.genai.prompt_sumarizacao', description='Initial version of summarization prompt', tags={'author': 'data-science-team@company.com', 'use_case': 'document_summarization', 'task': 'summarization', 'language': 'pt-br', 'model_compatibility': 'modelo', 'PromptVersionCount': '2'}>]